In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import io

c:\Users\sanit\OneDrive\Escritorio\TP integrador NN\TP-Integrador-RN\.venv\Lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.7'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
import mlflow
import mlflow.pytorch

In [3]:
mlflow.set_experiment("MLP_Clasificador_Imagenes")

2025/06/07 20:00:43 INFO mlflow.tracking.fluent: Experiment with name 'MLP_Clasificador_Imagenes' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/sanit/OneDrive/Escritorio/TP%20integrador%20NN/TP-Integrador-RN/mlruns/238164930765243842', creation_time=1749337243079, experiment_id='238164930765243842', last_update_time=1749337243079, lifecycle_stage='active', name='MLP_Clasificador_Imagenes', tags={}>

In [4]:
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as vutils

In [5]:
# Función para loguear una figura matplotlib en TensorBoard
def plot_to_tensorboard(fig, writer, tag, step):
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    image = Image.open(buf).convert("RGB")
    image = np.array(image)
    image = torch.tensor(image).permute(2, 0, 1) / 255.0
    writer.add_image(tag, image, global_step=step)
    plt.close(fig)

In [8]:
# Función para matriz de confusión y clasificación
def log_classification_report(model, loader, writer, step, prefix="val"):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    fig_cm, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=train_dataset.label_encoder.classes_)
    disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
    ax.set_title(f'{prefix.title()} - Confusion Matrix')

    # Guardar localmente y subir a MLflow
    fig_path = f"confusion_matrix_{prefix}_epoch_{step}.png"
    fig_cm.savefig(fig_path)
    mlflow.log_artifact(fig_path)
    os.remove(fig_path)

    plot_to_tensorboard(fig_cm, writer, f"{prefix}/confusion_matrix", step)

    cls_report = classification_report(all_labels, all_preds, target_names=train_dataset.label_encoder.classes_)
    writer.add_text(f"{prefix}/classification_report", f"<pre>{cls_report}</pre>", step)

    # También loguear texto del reporte
    with open(f"classification_report_{prefix}_epoch_{step}.txt", "w") as f:
        f.write(cls_report)
    mlflow.log_artifact(f.name)
    os.remove(f.name)


In [9]:
# Crear directorio de logs
log_dir = "runs/mlp_experimento_1"
writer = SummaryWriter(log_dir=log_dir)


In [10]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.image_paths = []
        self.labels = []

        class_names = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

        for cls in class_names:
            cls_dir = os.path.join(root_dir, cls)
            for fname in os.listdir(cls_dir):
                if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.image_paths.append(os.path.join(cls_dir, fname))
                    self.labels.append(cls)

        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        label = self.labels[idx]

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented["image"]

        return image, label

In [11]:
train_transform = A.Compose([
    A.Resize(64, 64),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(),
    ToTensorV2()
])


In [12]:
val_test_transform = A.Compose([
    A.Resize(64, 64),
    A.Normalize(),
    ToTensorV2()
])

In [13]:
# Paths
train_dir = r'skin-dataset-clasification/data/Split_smol/train/'
val_dir = r'skin-dataset-clasification/data/Split_smol/val/'

In [14]:
train_dataset = CustomImageDataset(train_dir, transform=train_transform)
val_dataset   = CustomImageDataset(val_dir, transform=val_test_transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)

In [15]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size=64*64*3, num_classes=10):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(train_dataset.labels))
model = MLPClassifier(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [17]:
# Entrenamiento y validación
def evaluate(model, loader, epoch=None, prefix="val"):
    log_classification_report(model, val_loader, writer, step=epoch, prefix="val")
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for i, (images, labels) in enumerate(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            loss_sum += loss.item()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Loguear imágenes del primer batch
            if i == 0 and epoch is not None:
                img_grid = vutils.make_grid(images[:8].cpu(), normalize=True)
                writer.add_image(f"{prefix}/images", img_grid, global_step=epoch)

    acc = 100.0 * correct / total
    avg_loss = loss_sum / len(loader)

    if epoch is not None:
        writer.add_scalar(f"{prefix}/loss", avg_loss, epoch)
        writer.add_scalar(f"{prefix}/accuracy", acc, epoch)

    return avg_loss, acc

In [18]:
# Loop de entrenamiento
n_epochs = 10
with mlflow.start_run():
    # Log hiperparámetros
    mlflow.log_params({
        "model": "MLPClassifier",
        "input_size": 64*64*3,
        "batch_size": batch_size,
        "lr": 1e-3,
        "epochs": n_epochs,
        "optimizer": "Adam",
        "loss_fn": "CrossEntropyLoss",
        "train_dir": train_dir,
        "val_dir": val_dir,
    })
for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = 100.0 * correct / total
    val_loss, val_acc = evaluate(model, val_loader, epoch=epoch, prefix="val")

    print(f"Epoch {epoch+1}:")
    print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")

    writer.add_scalar("train/loss", train_loss, epoch)
    writer.add_scalar("train/accuracy", train_acc, epoch)

    # Log en MLflow
    mlflow.log_metrics({
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc
    }, step=epoch)

Epoch 1/10: 100%|██████████| 22/22 [00:18<00:00,  1.21it/s]
c:\Users\sanit\OneDrive\Escritorio\TP integrador NN\TP-Integrador-RN\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\sanit\OneDrive\Escritorio\TP integrador NN\TP-Integrador-RN\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\sanit\OneDrive\Escritorio\TP integrador NN\TP-Integrador-RN\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being

Epoch 1:
  Train Loss: 2.8603, Accuracy: 23.42%
  Val   Loss: 2.1567, Accuracy: 40.00%


Epoch 2/10: 100%|██████████| 22/22 [00:21<00:00,  1.04it/s]
c:\Users\sanit\OneDrive\Escritorio\TP integrador NN\TP-Integrador-RN\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\sanit\OneDrive\Escritorio\TP integrador NN\TP-Integrador-RN\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\sanit\OneDrive\Escritorio\TP integrador NN\TP-Integrador-RN\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being

Epoch 2:
  Train Loss: 1.7352, Accuracy: 43.53%
  Val   Loss: 1.7658, Accuracy: 38.33%


Epoch 3/10: 100%|██████████| 22/22 [00:18<00:00,  1.20it/s]


...

Epoch 8:
  Train Loss: 0.9663, Accuracy: 62.79%
  Val   Loss: 1.3896, Accuracy: 50.00%


Epoch 9/10: 100%|██████████| 22/22 [00:14<00:00,  1.51it/s]


Epoch 9:
  Train Loss: 0.9810, Accuracy: 62.64%
  Val   Loss: 1.3722, Accuracy: 50.56%


Epoch 10/10: 100%|██████████| 22/22 [00:17<00:00,  1.29it/s]


Epoch 10:
  Train Loss: 0.8966, Accuracy: 64.51%
  Val   Loss: 1.2815, Accuracy: 55.56%


In [19]:
# Guardar modelo
torch.save(model.state_dict(), "mlp_model.pth")
print("Modelo guardado como 'mlp_model.pth'")
mlflow.log_artifact("mlp_model.pth")
mlflow.pytorch.log_model(model, artifact_path="pytorch_model")
print("Modelo guardado como 'mlp_model.pth'")

Modelo guardado como 'mlp_model.pth'


2025/06/07 20:08:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado como 'mlp_model.pth'


In [ ]:
!tensorboard --logdir=runs/mlp_experimento_1

http://localhost:6006 para ver el tensorboard